In [2]:
import cv2
from ultralytics import YOLO

# Load YOLO models
plate_detector = YOLO("best.pt")  # License plate detection model
char_recognizer = YOLO("model-m/weights/best.pt")  # Character recognition model

# Character Mapping
char_map = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'Metro', 11: 'A', 12: 'Bha', 13: 'Cha', 14: 'Chha', 15: 'Da', 16: 'DA', 17: 'E',
    18: 'Ga', 19: 'Gha', 20: 'Ha', 21: 'Ja', 22: 'Jha', 23: 'Ka', 24: 'Kha', 25: 'La',
    26: 'Ma', 27: 'Na', 28: 'Pa', 29: 'Sa', 30: 'Sha', 31: 'Ta', 32: 'THA', 33: 'Tha',
    34: 'U', 35: 'Bagerhat', 36: 'Bagura', 37: 'Bandarban', 38: 'Barguna', 39: 'Barisal',
    40: 'Bhola', 41: 'Brahmanbaria', 42: 'Chandpur', 43: 'Chapainawabganj', 44: 'Chatto',
    45: 'Chattogram', 46: 'Chuadanga', 47: 'Coxs Bazar', 48: 'Cumilla', 49: 'Dhaka',
    50: 'Dinajpur', 51: 'Faridpur', 52: 'Feni', 53: 'Gaibandha', 54: 'Gazipur',
    55: 'Gopalganj', 56: 'Habiganj', 57: 'Jamalpur', 58: 'Jessore', 59: 'Jhalokati',
    60: 'Jhenaidah', 61: 'Joypurhat', 62: 'Khagrachari', 63: 'Khulna', 64: 'Kishoreganj',
    65: 'Kurigram', 66: 'Kustia', 67: 'Lakshmipur', 68: 'Lalmonirhat', 69: 'Madaripur',
    70: 'Magura', 71: 'Manikganj', 72: 'Meherpur', 73: 'Moulvibazar', 74: 'Mymensingh',
    75: 'Naogaon', 76: 'Narail', 77: 'Narayanganj', 78: 'Narsingdi', 79: 'Natore',
    80: 'Netrokona', 81: 'Nilphamari', 82: 'Noakhali', 83: 'Pabna', 84: 'panchagarh',
    85: 'Patuakhali', 86: 'Pirojpur', 87: 'Raj', 88: 'Rajbari', 89: 'Rajshahi',
    90: 'Rangamati', 91: 'Rangpur', 92: 'Satkhira', 93: 'Shariatpur', 94: 'Sherpur',
    95: 'Sirajganj', 96: 'Sunamganj', 97: 'Sylhet', 98: 'Tangail', 99: 'Thakurgaon',
    100: 'Dha', 101: 'Ba'
}


In [30]:

# Open video stream
cap = cv2.VideoCapture("/home/avrohabib/Downloads/video/Video-data/v2.mp4")  # Use 0 for webcam, or provide a video file path

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit if video ends

    # Detect license plate
    plate_results = plate_detector(frame)

    for plate in plate_results:
        boxes = plate.boxes.xyxy

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            plate_img = frame[y1:y2, x1:x2]

            if plate_img.size == 0:
                continue  # Skip empty detections

            # Process detected license plate
            char_results = char_recognizer(plate_img)

            detected_chars = []
            for char in char_results:
                for cbox in char.boxes:
                    cx1, cy1, cx2, cy2 = cbox.xyxy[0]
                    class_id = int(cbox.cls)
                    center_x = (cx1 + cx2) / 2  # X center for sorting

                    detected_chars.append((center_x, class_id))

            # Separate letters and numbers
            label1, label2 = [], []
            for center_x, class_id in detected_chars:
                if class_id in range(0, 10):  # Numeric characters (0-9)
                    label2.append((center_x, class_id))
                else:  # Non-numeric characters
                    label1.append((center_x, class_id))

            # Sort characters by X position
            label1.sort(key=lambda x: x[0])
            label2.sort(key=lambda x: x[0])

            # Convert class IDs to text
            sorted_text = "".join([char_map[c[1]] for c in label1]) + " " + "".join([char_map[c[1]] for c in label2])

            # Draw bounding box around license plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, sorted_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Real-Time License Plate Recognition", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [28]:
import cv2
import torch
from ultralytics import YOLO
import time

# Load YOLO models
plate_detector = YOLO("best.pt")  # License plate detection model
char_recognizer = YOLO("model-m/weights/best.pt")  # Character recognition model



# Open webcam or video file
cap = cv2.VideoCapture("/home/avrohabib/Downloads/video/Video-data/v1.mp4")  # 0 for webcam, or replace with video file path

# Set higher FPS for smooth video
cap.set(cv2.CAP_PROP_FPS, 60)  
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 416)  # Reduce resolution for speed
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 416)

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
plate_detector.to(device)
char_recognizer.to(device)

frame_skip = 5  # Skip every 2nd frame for speed
frame_count = 0

while cap.isOpened():
    start_time = time.time()  # Track FPS
    ret, frame = cap.read()
    if not ret:
        break  # Exit if video ends

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Skip frames to improve FPS

    # Detect license plate
    plate_results = plate_detector(frame, conf=0.6, imgsz=416, verbose=False)   # Set confidence threshold

    for plate in plate_results:
        boxes = plate.boxes.xyxy

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            plate_img = frame[y1:y2, x1:x2]

            if plate_img.size == 0:
                continue  # Skip empty detections

            # Process detected license plate
            char_results = char_recognizer(plate_img, conf=0.6, imgsz=320, verbose=False)

            detected_chars = []
            for char in char_results:
                for cbox in char.boxes:
                    cx1, cy1, cx2, cy2 = cbox.xyxy[0]
                    class_id = int(cbox.cls)
                    center_x = (cx1 + cx2) / 2  # X center for sorting

                    detected_chars.append((center_x, class_id))

            # Separate and sort letters/numbers
            label1, label2 = [], []
            for center_x, class_id in detected_chars:
                if class_id in range(0, 10):  # Numeric characters
                    label2.append((center_x, class_id))
                else:  # Letters
                    label1.append((center_x, class_id))

            label1.sort(key=lambda x: x[0])  # Sort letters
            label2.sort(key=lambda x: x[0])  # Sort numbers

            # Convert class IDs to text
            sorted_text = "".join([char_map[c[1]] for c in label1]) + " " + "".join([char_map[c[1]] for c in label2])

            # Draw bounding box and text
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, sorted_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Calculate and display FPS
    fps = 1 / (time.time() - start_time)
    cv2.putText(frame, f'FPS: {int(fps)}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show processed video
    cv2.imshow("Real-Time License Plate Recognition", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import torch
import threading
import time
from ultralytics import YOLO

# Load YOLO models (Optimized for GPU)
# Load YOLO models
plate_detector = YOLO("best.pt")  # License plate detection model
char_recognizer = YOLO("model-m/weights/best.pt")  # Character recognition model




# Capture video with threading
cap = cv2.VideoCapture("/home/avrohabib/Downloads/video/Video-data/v3.mp4")
cap.set(cv2.CAP_PROP_FPS, 60)  # Set high FPS
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 416)  # Smaller frame for speed
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 416)

frame = None
running = True

def capture_frames():
    global frame, running
    while running:
        ret, temp_frame = cap.read()
        if ret:
            frame = temp_frame

# Start capture thread
threading.Thread(target=capture_frames, daemon=True).start()

frame_skip = 2  # Dynamically adjust this if FPS is too low
frame_count = 0

while cap.isOpened():
    if frame is None:
        continue  # Skip if no frame available

    start_time = time.time()
    frame_count += 1

    if frame_count % frame_skip != 0:
        continue  # Skip frames to boost FPS

    # Detect license plate (Set confidence to 0.6)
    plate_results = plate_detector(frame, conf=0.6, imgsz=416, verbose=False)  

    for plate in plate_results:
        boxes = plate.boxes.xyxy

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            plate_img = frame[y1:y2, x1:x2]

            if plate_img.size == 0:
                continue  # Skip empty detections

            # Process detected license plate
            char_results = char_recognizer(plate_img, conf=0.6, imgsz=320, verbose=False)

            detected_chars = []
            for char in char_results:
                for cbox in char.boxes:
                    cx1, cy1, cx2, cy2 = cbox.xyxy[0]
                    class_id = int(cbox.cls)
                    center_x = (cx1 + cx2) / 2  # X center for sorting

                    detected_chars.append((center_x, class_id))

            # Separate and sort letters/numbers
            label1, label2 = [], []
            for center_x, class_id in detected_chars:
                if class_id in range(0, 10):  # Numeric characters
                    label2.append((center_x, class_id))
                else:  # Letters
                    label1.append((center_x, class_id))

            label1.sort(key=lambda x: x[0])  # Sort letters
            label2.sort(key=lambda x: x[0])  # Sort numbers

            # Convert class IDs to text
            sorted_text = "".join([char_map[c[1]] for c in label1]) + " " + "".join([char_map[c[1]] for c in label2])

            # Draw bounding box and text
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, sorted_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Calculate FPS
    fps = 1 / (time.time() - start_time)
    cv2.putText(frame, f'FPS: {int(fps)}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Adjust frame skipping dynamically
    if fps < 15: 
        frame_skip = 3  # Skip more frames if slow
    elif fps > 30: 
        frame_skip = 1  # Process every frame if fast

    # Show output
    cv2.imshow("Real-Time License Plate Recognition", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        running = False
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'char_map' is not defined

: 